In [1]:
# from zipfile import ZipFile

# with ZipFile('compressed.zip', 'r') as zf:
#     zf.extractall('.')

In [2]:
# !pip install torchinfo
# !pip install pandas
# !pip install matplotlib
# !pip install scipy

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchinfo import summary
from torchvision import transforms

from models.Discriminator import *
from models.Encoder import *
from models.Generator import *
from loss_functions import *
from DAMSM_trainer import *

import dataset
import os
import matplotlib.pyplot as plt
import config.settings as config

SyntaxError: '(' was never closed (DAMSM_trainer.py, line 79)

In [ ]:
rnn_model = RNN_Encoder(config.WORD_SIZE, number_hidden=config.EMBEDDING_DIM)

rnn_model.load_state_dict(torch.load("saved_models/rnn_model_state_dict.pt"))

generator = GenerativeNetwork()

generator.load_state_dict(torch.load("saved_models/generator_state_dict.pt"))

rnn_model.eval()
generator.eval()

In [ ]:
import re
config.DEVICE = torch.device('cpu')
input_sentence = input("Write your input sentence: ").lower()
input_sentence = re.sub('[^a-zA-Z0-9]+', ' ', input_sentence)
input_sentence = list(map(lambda word: dataset.rev_text_dict[word], input_sentence.split()))

input_len = torch.tensor(len(input_sentence)).unsqueeze(0)
input_sentence = [np.pad(input_sentence, 
                       (0, config.SENTENCE_SIZE - input_len.squeeze())
                       )]

hidden = rnn_model.init_hidden(1)

input_sentence = torch.tensor(input_sentence)

word_emb, sen_emb = rnn_model(input_sentence.long(), input_len, hidden)

noise = torch.randn((1, config.Z_DIM))

mask = (input_sentence == 0)
num_words = word_emb.size(2)
if mask.size(1) > num_words:
    mask = mask[:, :num_words]

img = generator(noise, sen_emb, word_emb, mask)[0]

plt.imshow(img[-1].squeeze().permute(1, 2, 0).detach().numpy())